In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import special
from scipy import stats


import h5py

def cdf_from_pdf(x, pdf):
    cdf = []
    for i in range(x.size):
        cdf.append(np.trapz(pdf[0:i+1], x[0:i+1]))
    return np.array(cdf)

path2results = Path.home()/'group'/'project'/'results'/'machine_learning'\
/'dnn'/'triggering'/'deep_filter_paper'/'eval'

#path2results = Path.home()/'group'/'project'/'results'/'machine_learning'\
#/'dnn'/'triggering'/'eval_outputs'/'signal_grid'

name = '220624_86.9to87.0deg_5mm_test_output_290epoch.npz'


In [ ]:
##result_file = h5py.File(path2results/name)
result_file = np.load(path2results/name)

#output = special.log_softmax(result_file['output'][:], axis=-1)
output = result_file['output'][:]
labels = result_file['labels'][:]

In [ ]:
print(special.softmax(output[0:20, :], axis=-1))
print(labels[0:20])

In [ ]:
np.unique(labels)

In [ ]:
output_11 = output[:, 1][np.where(labels==1)[0]] # TP
output_10 = output[:, 1][np.where(labels==0)[0]] # FP
output_1_1 = output[:, 1][np.where(labels==-1)[0]] # ??

output_00 = output[:, 0][np.where(labels==0)[0]] # TN
output_01 = output[:, 0][np.where(labels==1)[0]] # FN

sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

#x_bins = np.linspace(0,1, 1001)
x_bins = np.linspace(-15,15,2001)

hist1 = ax.hist(output_00, x_bins, histtype='step', density=True)
hist0 = ax.hist(output_01, x_bins, histtype='step', density=True)

ax.set_yscale('log')

pdf_1 = hist1[0]
pdf_0 = hist0[0]

cdf_1 = cdf_from_pdf(x_bins[1:], pdf_1)
cdf_0 = cdf_from_pdf(x_bins[1:], pdf_0)

sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(x_bins[1:], 1-cdf_1)
ax.plot(x_bins[1:], 1-cdf_0)

sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(1-cdf_0, 1-cdf_1)
ax.plot(np.linspace(0, 1, 10001), np.linspace(0, 1, 10001), )
ax.set_xscale('log')
xlims = ax.get_xlim()
ax.set_xlim(1e-4, xlims[1])



In [ ]:
sample = special.softmax(result_file['0']['0_x'][:], axis=-1)

In [ ]:
sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

clist = sns.color_palette('bright', n_colors=10)

test_i = 10

ax.hist(sample[:, test_i, 1], bins=30, histtype='step', density=True, color=clist[0])
ax.hist(sample[:, test_i, 0], bins=30, histtype='step', density=True, color=clist[1])


mean_1, std_1 = stats.norm.fit(sample[:, test_i, 1],)
x = np.linspace(0, 1, 1001)
pdf_1 = stats.norm.pdf(x, loc=mean_1, scale=std_1)
ax.plot(x, pdf_1, color=clist[0])

mean_0, std_0 = stats.norm.fit(sample[:, test_i, 0],)
x = np.linspace(0, 1, 1001)
pdf_0 = stats.norm.pdf(x, loc=mean_0, scale=std_0)
ax.plot(x, pdf_0, color=clist[1])

print(result_file['0']['0_y'][test_i])

In [ ]:
means_0, stds_0, means_1, stds_1, labels = [], [], [], [], []

for ep in result_file.keys():
    n_batch = len(result_file[ep]) // 2
    
    for batch in range(n_batch):
        
        model_output = special.softmax(result_file[ep][f'{batch}_x'][:], axis=-1)
        label = result_file[ep][f'{batch}_y'][:]
        
        for i in range(model_output.shape[1]):
            
            mean_0, std_0 = stats.norm.fit(model_output[:, i, 0])
            mean_1, std_1 = stats.norm.fit(model_output[:, i, 1])
            
            means_0.append(mean_0)
            means_1.append(mean_1)
            stds_0.append(std_0)
            stds_1.append(std_1)
            labels.append(int(label[i]))
            
means_0 = np.array(means_0)
means_1 = np.array(means_1)
stds_0 = np.array(stds_0)
stds_1 = np.array(stds_1)
labels = np.array(labels)
            
            

In [ ]:
means_00 = means_0[np.where(labels==0)[0]]
means_11 = means_1[np.where(labels==1)[0]] # TP
means_01 = means_0[np.where(labels==1)[0]]
means_10 = means_1[np.where(labels==0)[0]] # FP

In [ ]:
sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

x_bins = np.linspace(0,1, 1001)

hist1 = ax.hist(means_11, x_bins, histtype='step', density=True)
hist0 = ax.hist(means_10, x_bins, histtype='step', density=True)

pdf_1 = hist1[0]
pdf_0 = hist0[0]

cdf_1 = cdf_from_pdf(x_bins[1:], pdf_1)
cdf_0 = cdf_from_pdf(x_bins[1:], pdf_0)

sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(x_bins[1:], 1-cdf_1)
ax.plot(x_bins[1:], 1-cdf_0)

sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(1-cdf_0, 1-cdf_1)
ax.plot(x_bins, x_bins, )
ax.set_xscale('log')
xlims = ax.get_xlim()
ax.set_xlim(1e-4, xlims[1])



In [ ]:
sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

hist = ax.hist(signal_output[:, 1], 101)
hist = ax.hist(noise_output[:, 0], 101)

In [ ]:
sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

bins = np.linspace(-1, 8, 1001)

hist_sig = ax.hist(signal_output[:, 1], bins, density=True, histtype='step')
hist_noise = ax.hist(noise_output[:, 1], bins, density=True, histtype='step')

In [ ]:
x_sig = hist_sig[1][1:]
x_noise = hist_noise[1][1:]

pdf_sig = hist_sig[0]
pdf_noise = hist_noise[0]

cdf_sig = cdf_from_pdf(x_sig, pdf_sig)
cdf_noise = cdf_from_pdf(x_noise, pdf_noise)

sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(x_sig, pdf_sig)
ax.plot(x_noise, pdf_noise)

sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(x_sig, 1-cdf_sig)
ax.plot(x_noise, 1-cdf_noise)


sns.set_theme(context='talk', style='whitegrid', palette='bright')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(1-cdf_noise, 1-cdf_sig)
ax.plot(np.linspace(0, 1, 1001), np.linspace(0, 1, 1001), '--')
ax.set_xscale('log')

xlims = ax.get_xlim()

ax.set_xlim(1e-4, xlims[1])
